In [3]:
import pickle
import pandas as pd
import networkx as nx
from glob import glob

In [2]:
folder = "../data/mgp_data/"

In [80]:
def load_hindex():
    filenames     = glob("./mgp_author_h-index*.pkl")
    hindex_df     = pd.read_csv(folder+"mgp_h-index.csv")
    hindex_df     = hindex_df[~(hindex_df["h_index"].isnull())].copy()
    hindex_dict   = dict(zip(hindex_df["mgp_id"], hindex_df["h_index"]))
    print(f"Initial Count: {len(hindex_dict)}")
    data_list = []
    for filename in filenames:
        with open(filename, "rb") as f:
            data = pickle.load(f)
        data = [(k,v["hindex"]) for k,v in data.items() if v is not None]
        #print(data)
        data_list = data_list+data
    hindex_dict.update(dict(data_list))
    print(f"updated Count: {len(hindex_dict)}")
    return hindex_dict

In [3]:
def load_pickle(filename, folder= "../data/mgp_data/"):
    with open(folder+filename,"rb") as fp:
        data =  pickle.load(fp)
        data_dict = dict(list(data))
    return data_dict

In [4]:
def get_metrics():
    #hindex_df     = pd.read_csv(folder+"mgp_h-index.csv")
    mathscinet     = pd.read_csv(folder+"mod_info_mathscinet.csv")
    hindex = load_hindex()
    mathscinet["processed_publ_area"] = mathscinet["publ_by_area"].apply(lambda x :[each_publ[-1] for each_publ in eval(x)] if type(x) == str else x )
    mgpid2citation_count  = dict(zip(mathscinet["mgp_id"],mathscinet["total_citation_count"]))
    mgpid2publ_count      = dict(zip(mathscinet["mgp_id"],mathscinet["total_publ_count"]))
    mgpid2_name_variation = dict(zip(mathscinet["mgp_id"],mathscinet["published_as"]))
    mgpid2_publ_area      = dict(zip(mathscinet["mgp_id"],mathscinet["processed_publ_area"]))
    return (hindex,mgpid2citation_count,mgpid2publ_count,mgpid2_name_variation,mgpid2_publ_area)

In [5]:
def get_graph_metrics():
    graph = nx.read_gpickle("topic_co_occurance_graph_80.gpickle")
    pr = nx.pagerank(graph, alpha=0.85)
    bc = nx.betweenness_centrality(graph)
    return (pr, bc)

In [ ]:
if __name__=="__main__":
    pr, bc = get_graph_metrics()
    hindex,mgpid2citation_count,mgpid2publ_count,mgpid2_name_variation,mgpid2_publ_area = get_metrics()